In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import sys
import pandas as pd
import urllib.request
import math
import gpytorch
from matplotlib import pyplot as plt
import random
import numpy as np
import urllib.request
import os
from math import floor
import black_box
import tqdm

In [2]:
data_dim = 102

class FCN(torch.nn.Sequential):
    def __init__(self):
        super(FCN, self).__init__()
        self.add_module('linear1', torch.nn.Linear(data_dim, 50))
        self.add_module('relu1', torch.nn.ReLU())
        self.add_module('linear2', torch.nn.Linear(50, 2))

feature_extractor = FCN()

class GPRegressionModel(gpytorch.models.ExactGP):
        def __init__(self, train_x, train_y, likelihood):
            super(GPRegressionModel, self).__init__(train_x, train_y, likelihood)
            self.mean_module = gpytorch.means.ConstantMean()
            self.covar_module = gpytorch.kernels.GridInterpolationKernel(
                gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel(ard_num_dims=2)),
                num_dims=2, grid_size=100
            )
            self.feature_extractor = feature_extractor

            # This module will scale the NN features so that they're nice values
            self.scale_to_bounds = gpytorch.utils.grid.ScaleToBounds(-1., 1.)

        def forward(self, x):
            # We're first putting our data through a deep net (feature extractor)
            projected_x = self.feature_extractor(x)
            projected_x = self.scale_to_bounds(projected_x)  # Make the NN values "nice"

            mean_x = self.mean_module(projected_x)
            covar_x = self.covar_module(projected_x)
            return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)


def train(train_x, train_y, num_device=0):
    device = torch.device(f'cuda:{num_device}' if torch.cuda.is_available() else 'cpu')
    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    model = GPRegressionModel(train_x, train_y, likelihood)

    model.to(device)
    likelihood.to(device)

    training_iterations = 60

    optimizer = torch.optim.Adam([
    {'params': model.feature_extractor.parameters()},
    {'params': model.covar_module.parameters()},
    {'params': model.mean_module.parameters()},
    {'params': model.likelihood.parameters()},], lr=0.01)
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

    demo_parity = 0
    iterator = range(training_iterations)
    
    model.train()
    likelihood.train()
    
    best_loss = 10e5
    penalty_count = 0
    for i in tqdm.tqdm(range(training_iterations)):
#         print(i)
        # Zero backprop gradients
        optimizer.zero_grad()
        # Get output from model
        output = model(train_x.to(device))

        # Calc loss and backprop derivatives
        loss = -mll(output, train_y.to(device))
        
        if loss >= best_loss:
            penalty_count += 1
        else:
            penalty_count = 0
            best_loss = loss
        
        if penalty_count == 5:
            break
        
        loss.backward()
        optimizer.step()


        #     model.eval()
        #     likelihood.eval()

    return model, likelihood


In [3]:
df_base = pd.read_csv("Data/data1.csv")

In [4]:
df_base.drop(['Unnamed: 0'], axis=1, inplace = True)
df_base_pos = df_base[df_base['gender_Female'] > 0]
df_base_neg = df_base[df_base['gender_Female'] <= 0]

In [5]:
df_base_neg = df_base_neg.reset_index(drop=True)
df_base_pos = df_base_pos.reset_index(drop=True)

In [6]:
temp_pos = df_base_pos.sample(1000)
temp_neg = df_base_neg.sample(1000)
pos_queried, pos_labels = temp_pos.drop(columns='income').values, temp_pos['income'].values
neg_queried, neg_labels = temp_neg.drop(columns='income').values, temp_neg['income'].values

In [7]:
#### training the GPs ####
model_pos, likelihood_pos = train(
    torch.tensor(pos_queried.astype(np.float32)),
    torch.tensor(pos_labels.astype(np.float32)),
    num_device=0
)

model_neg, likelihood_neg = train(
    torch.tensor(neg_queried.astype(np.float32)),
    torch.tensor(neg_labels.astype(np.float32)),
    num_device=0
)


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 60/60 [00:05<00:00, 11.83it/s]


In [8]:
model_pos.eval()
likelihood_pos.eval()

model_neg.eval()
likelihood_neg.eval()

GaussianLikelihood(
  (noise_covar): HomoskedasticNoise(
    (raw_noise_constraint): GreaterThan(1.000E-04)
  )
)

In [9]:
from vae import VAE

In [18]:
vae1 = VAE(102, 64, 16).to(device)
checkpoint1 = torch.load('checkpoints/vae1.pth')

vae0 = VAE(102, 64, 16).to(device)
checkpoint0 = torch.load('checkpoints/vae0.pth')

In [19]:
vae0.load_state_dict(checkpoint0['model_state_dict'])
vae0.eval()

VAE(
  (encoder): Encoder(
    (fc1): Linear(in_features=102, out_features=64, bias=True)
    (fc2): Linear(in_features=64, out_features=64, bias=True)
    (fc3): Linear(in_features=64, out_features=32, bias=True)
  )
  (decoder): Decoder(
    (fc1): Linear(in_features=16, out_features=64, bias=True)
    (fc2): Linear(in_features=64, out_features=64, bias=True)
    (fc3): Linear(in_features=64, out_features=102, bias=True)
  )
)

In [20]:
vae1.load_state_dict(checkpoint1['model_state_dict'])
vae1.eval()

VAE(
  (encoder): Encoder(
    (fc1): Linear(in_features=102, out_features=64, bias=True)
    (fc2): Linear(in_features=64, out_features=64, bias=True)
    (fc3): Linear(in_features=64, out_features=32, bias=True)
  )
  (decoder): Decoder(
    (fc1): Linear(in_features=16, out_features=64, bias=True)
    (fc2): Linear(in_features=64, out_features=64, bias=True)
    (fc3): Linear(in_features=64, out_features=102, bias=True)
  )
)

In [ ]:
data_X_0, data_Y_0 = [torch.FloatTensor(neg_queried).to(device),], [torch.FloatTensor(neg_labels).to(device),]
data_X_1, data_Y_1 = [torch.FloatTensor(pos_queried).to(device),], [torch.FloatTensor(pos_labels).to(device),]

num_device = 0
device = torch.device(f'cuda:{num_device}' if torch.cuda.is_available() else 'cpu')

query_budget = 5000
my_blackbox = black_box.BlackBox('checkpoints/blackbox_preaudit.pth')

for i in range(query_budget // 2 - 1000):
    print(i)
    x_0 = torch.normal(0., 1., size = (1, 102), requires_grad=True, device=device)
    sample_size = 20
    X_0 = x_0
    X_0.repeat(sample_size, 1)
    
    optimizer = torch.optim.AdamW((x_0,), lr=1e-2)
    optimizer.zero_grad()
    n_iters= 50
    
    best_loss = 10e5
    penalty_count = 0
    for j in tqdm.tqdm(range(n_iters)):
        X_sample_0, _, _ = vae0(X_0)
        var = likelihood_neg(model_neg(X_sample_0)).variance
        loss = (-var).mean()
        
        if loss >= best_loss:
            penalty_count += 1
        else:
            penalty_count = 0
            best_loss = loss
            
        if penalty_count == 5:
            break
        
        loss.backward()
        optimizer.step()
    print(var)

    _, y_0 = my_blackbox.query(x_0)

    data_X_0.append(x_0)
    data_Y_0.append(y_0.flatten().to(device))
    
    X_train = torch.cat(data_X_0, 0)
    Y_train = torch.cat(data_Y_0, 0).flatten()
    model_neg, likelihood_neg = train(X_train, Y_train)
    
    model_neg.eval()
    likelihood_neg.eval()
    
    x_1 = torch.normal(0., 1., size = (1, 102), requires_grad=True, device=device)
    sample_size = 20
    X_1 = x_1
    X_1.repeat(sample_size, 1)

    optimizer = torch.optim.AdamW((x_1,), lr=1e-2)
    optimizer.zero_grad()
    
    best_loss = 10e5
    penalty_count = 0
    for j in tqdm.tqdm(range(n_iters)):
        X_sample_1, _, _ = vae1(X_1)
        var = likelihood_pos(model_pos(X_sample_1)).variance
        loss = (-var).mean()
        
        if loss >= best_loss:
            penalty_count += 1
        else:
            penalty_count = 0
            best_loss = loss
            
        if penalty_count == 5:
            break

        loss.backward()
        optimizer.step()
    print(var)

    _, y_1 = my_blackbox.query(x_1)
    data_X_1.append(x_1)
    data_Y_1.append(y_1.flatten().to(device))

    model_pos, likelihood_pos = train(torch.cat(data_X_1, 0), torch.cat(data_Y_1, 0).flatten())
    
    model_pos.eval()
    likelihood_pos.eval()

0


 10%|███████████▌                                                                                                        | 5/50 [00:00<00:05,  8.15it/s]


tensor([0.4363], device='cuda:0', grad_fn=<ExpandBackward0>)


 10%|███████████▌                                                                                                        | 5/50 [00:00<00:04, 10.70it/s]


tensor([1.0000e-06], device='cuda:0', grad_fn=<ClampMinBackward0>)


...

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 60/60 [00:20<00:00,  2.97it/s]


1040


 20%|███████████████████████                                                                                            | 10/50 [00:01<00:06,  6.57it/s]


tensor([0.4351], device='cuda:0', grad_fn=<ExpandBackward0>)


 22%|████████████████████████▉                                                                                          | 13/60 [00:03<00:11,  3.92it/s]

In [ ]:
X_dem_pos = torch.normal(0., 1., size = (1000, 102), device = device)
X_dem_neg = torch.normal(0., 1., size = (1000, 102), device = device)

X_dem_pos = vae1(X_dem_pos)
X_dem_neg = vae0(X_dem_neg)

Y_dem_pos = model_pos(X_dem_pos).mean
Y_dem_neg = model_neg(X_dem_neg).mean

demo_parity = torch.abs(Y_dem_pos.mean() - Y_dem_neg.mean())

print(demo_parity)

In [ ]:
torch.save(demo_parity, 'demo_parity.pt')

In [ ]:
torch.save(model_pos.state_dict(), 'model_pos.pth')
torch.save(model_neg.state_dict(), 'model_neg.pth')

In [20]:
torch.FloatTensor(torch.Tensor([0.1]) >= 1)

TypeError: expected TensorOptions(dtype=float, device=cpu, layout=Strided, requires_grad=false (default), pinned_memory=false (default), memory_format=(nullopt)) (got TensorOptions(dtype=bool, device=cpu, layout=Strided, requires_grad=false (default), pinned_memory=false (default), memory_format=(nullopt)))

In [15]:
x_random = torch.normal(0.,1.,size=(100,102), requires_grad=True)
optimizer = torch.optim.AdamW((x_random,), lr=0.001)

In [16]:
x_sampled, _, _ = vae1(x_random)

In [17]:
num_device = 0
device = torch.device(f'cuda:{num_device}' if torch.cuda.is_available() else 'cpu')

In [18]:
var = likelihood_pos(model_pos(x_sampled.to(device))).variance

In [19]:
loss = (-var).mean()

In [20]:
optimizer.zero_grad()
loss.backward()
optimizer.step()

In [21]:
x_sampled

tensor([[-1.2096e-01,  1.8022e-02, -7.2392e-02,  ...,  3.7889e-04,
          1.7755e-03,  1.0177e+00],
        [-1.1681e-01,  2.3465e-02, -7.0598e-02,  ...,  2.2722e-03,
          1.0722e-03,  9.9750e-01],
        [-1.2206e-01,  1.3110e-02, -6.8568e-02,  ...,  1.1457e-03,
          1.9830e-03,  1.0022e+00],
        ...,
        [-1.1342e-01,  1.9737e-02, -6.8200e-02,  ...,  3.1049e-03,
          1.0210e-03,  9.8822e-01],
        [-1.1880e-01,  1.8173e-02, -6.6933e-02,  ...,  3.3376e-03,
          7.8000e-04,  9.9166e-01],
        [-1.1915e-01,  1.6072e-02, -6.9233e-02,  ...,  3.0772e-03,
          9.1844e-04,  9.9567e-01]], grad_fn=<AddmmBackward0>)